#   <center>Machine Learning Engineer Nanodegree</center>
##   <center>Project: Dogs vs. Cats Redux: Kernels Edition</center>
<center>
Author: Kyle Chen<br>
Date: 20180506<br>
Version: 20180506v1
</center>

---

###   写在前面
-   这次的毕业项目选做猫狗, 一方面是因为资料比较多, 资源比较丰富. 另外一方面, 图形识别也是当下的一大热点. 虽然毕业后可能会继续从事传统机器学习方面的研究, 但是能够有这段经历还是很不错的, 也能丰富自己的简历与深度学习的知识.
-   在这个项目中, 将通过评估几种模型与不同程度的调优, 来不断优化我们的模型框架, 以达到最终的top 10%标准.

###   准备数据
-   本文中的DataSet是已经从kaggle上拖取到本地, 并放入当前工作目录中, 但是由于Github的大小限制, 如果你想要执行并研究其中的一些代码, 请自行准备好数据集, 并存放到./DataSet目录下. kaggle传送门 <a href='https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data'>dogs-vs-cats-redux-kernels-edition/data</a>.
-   或者你可以直接运行下框的代码创建目录结构与拖取数据.

In [ ]:
# 如果你已经拖取了数据, 请勿执行此代码框中的代码
!mkdir DataSet
!wget -c "https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1525872567&Signature=Vp1XJm2t%2FeDE9TqhWRHhhDuCD7GmOw4LhuU9cVU%2FNKbur08UKSw8UkDRm%2B6quFq0NL41vn%2BA45YkXvwlmiyM%2Br51%2BvXpWUHtYi3XAMwxjEVn7HI7dwyEP2tSO1H9SS%2Bi5YM8e94zNQ5mrpUypxL52HDBH0BVJBGs40RFR7uAiSeLizUNwArPl5zyP11EkOPrcFC2umd8e5BmfxpfRWUNTwr4%2FpfN6AAu%2BOsPU3QakCnzqxYQ1idOsQ4AO4AecseLYtEdeXJaov0lwaUVh9BIRMZibW4Sylh9RW0QmTspNbeCdZ%2BiTzMMfHxII5DhuXznZcpHOLRIG7%2F8XqOULoMzAA%3D%3D" -O DataSet/train.zip
!wget -c "https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/test.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1525872601&Signature=aHXH%2B1qiQOk5ny9b3YrivSZQB23neGcHxVubwp6olX8SPz6V5wfkqpbs2ncy%2B%2BLozRBLx%2BG86KdjqsuGuo%2FbYYjMwwh%2F1784dKFaNlFxBR3x8jIn6ji221MWwCkX9Cij20xC9ECpaXWBap8jYypRlAp%2Ff8AlTJF1zY8xQ84su8Gs2y8tDVs9Gt0OuiKu4dNJ017ZPjclPYNjm2%2BCG1GcpgCmZy6qkqvW%2FsuMPr%2BLcGFB1X0xrqYLxmX1JJGlikoZ%2FjQiJ5ZYjIhnLm05BhWdegChS24hnDyF4Mo4DoI9r9NBpRIPqF3kW%2BSZ0ci%2FRgutEvqr7OXcuRpOIR4pPYVZdw%3D%3D" -O DataSet/test.zip
!unzip test/train.zip -d DataSet/
!unzip test/test.zip -d DataSet/

###   关于DataSet
-   先了解下数据集的组成.

In [24]:
!ls -ahl DataSet/

total 109136
drwxr-xr-x      7 Kyle  staff   224B May  7 21:33 .
drwxr-xr-x     10 Kyle  staff   320B May  7 21:32 ..
-rw-r--r--      1 Kyle  staff   111K May  7 15:08 sample_submission.csv
drwxr-xr-x  12502 Kyle  staff   391K May  7 15:08 test
-rw-r--r--@     1 Kyle  staff   3.7M May  7 21:33 test.zip
drwxr-xr-x  25002 Kyle  staff   781K May  7 15:08 train
-rw-r--r--@     1 Kyle  staff    49M May  7 21:33 train.zip


-   test目录下存放的是kaggle准备好的测试集.
-   train目录下存放的是训练集, 当然, 还需要将其细分为训练集与验证集两个部分.

In [25]:
!ls -ahl DataSet/train/ | head -n 5

total 1217008
drwxr-xr-x  25002 Kyle  staff   781K May  7 15:08 .
drwxr-xr-x      7 Kyle  staff   224B May  7 21:33 ..
-rw-r--r--      1 Kyle  staff    12K May  7 15:08 cat.0.jpg
-rw-r--r--      1 Kyle  staff    16K May  7 15:08 cat.1.jpg


-   发现数据集命名是有规则的, 遵循label.n.jpg的原则.

In [26]:
!echo "cats | $(find DataSet/train/ -name 'cat*' | wc -l)"
!echo "dogs | $(find DataSet/train/ -name 'dog*' | wc -l)"

cats |    12500
dogs |    12500


-   发现cats/dogs样本类型分布均匀.

###   Import Libs

In [37]:
# 导入我们后面需要用到的库
import os
import re
import numpy as np
from PIL import Image
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint 

###   Initial Global Variables

In [38]:
# initial global variables
TRAIN_DIR = 'DataSet/train'
TEST_DIR = 'DataSet/test'

###   导入数据集

In [39]:
# load data from TRAIN_DIR
def load_data():
    img_list = os.listdir(TRAIN_DIR)
    nums = len(img_list)
    data = np.empty((nums, 224, 224, 3), dtype="float32")
    label = np.empty((nums, ))
    
    i = 0
    while i < nums:
        img = img_list[i]
        imgObj = Image.open("{}/{}".format(TRAIN_DIR, img))
        arr = np.asarray(imgObj, dtype="float32")
        arr.resize((224, 224, 3))
        data[i, :, :, :] = arr
        
        if re.match(r'^cat\.', img) != None:
            label[i] = 0
            
        elif re.match(r'^dog\.', img) != None:
            label[i] = 1            
        i += 1
    return(data, label)

In [40]:
# run load data
data, label = load_data()

-   CNN模型框架设计

In [41]:
cnn_model = Sequential()
shape_input = (len(data[0]), len(data[0][0]), len(data[0][0][0]))
cnn_model.add(Conv2D(filters=16, kernel_size=2, input_shape=shape_input))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(Dense(133, activation='relu'))
cnn_model.add(Conv2D(filters=32, kernel_size=2))
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(Dense(133, activation='relu'))
cnn_model.add(Conv2D(filters=64, kernel_size=2))
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(GlobalAveragePooling2D(dim_ordering='default'))
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 223, 223, 16)      208       
_________________________________________________________________
batch_normalization_8 (Batch (None, 223, 223, 16)      64        
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
dense_17 (Dense)             (None, 111, 111, 133)     2261      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 110, 110, 32)      17056     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 55, 55, 32)        0         
_________________________________________________________________
dense_18 (Dense)             (None, 55, 55, 133)       4389      
__________

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `GlobalAveragePooling2D` call to the Keras 2 API: `GlobalAveragePooling2D(data_format=None)`
  if sys.path[0] == '':


-   编译模型

In [35]:
## 编译模型
cnn_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

-   训练CNN模型

In [36]:
epochs = 5
batch_size = 20
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.cnn.hdf5', 
                                   verbose=1, save_best_only=True)
cnn_model.fit(data, label, validation_split = 0.3,
                epochs = epochs, batch_size = batch_size, shuffle = True,
                callbacks=[checkpointer], verbose=1)

Train on 17500 samples, validate on 7500 samples
Epoch 1/5
15580/17500 [=========================>....] - ETA: 2:11 - loss: 0.7010 - acc: 0.5418

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-26d49c01d04f>", line 7, in <module>
    callbacks=[checkpointer], verbose=1)
  File "/anaconda3/lib/python3.6/site-packages/keras/models.py", line 963, in fit
    validation_steps=validation_steps)
  File "/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1705, in fit
    validation_steps=validation_steps)
  File "/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1235, in _fit_loop
    outs = f(ins_batch)
  File "/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2478, in __call__
    **self.session_kwargs)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 778, in run
    run_metadata_ptr)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/clien

KeyboardInterrupt: 

In [ ]:
cnn_predictions = cnn_model.predict(data, batch_size = 32)
cnn_predictions = [np.argmax(pre) for pre in cnn_predictions]

In [19]:
# 报告测试准确率
test_accuracy = 100*np.sum(np.array(cnn_predictions)==np.argmax(label))/len(cnn_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 100.0000%
